# Dependencies

In [1]:
import tensorflow as tf
import numpy as np
import os
from collections import Counter
from matplotlib import pyplot as plt

/Users/d062280/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import sys
sys.path.append('/usr/local/opt/opencv/lib//python3.6/site-packages/')
import cv2

# Load Data

In [3]:
def get_rec_files(path,ext):
    content = os.listdir(path)
    if len(content) == 0:
        return []
    if ".DS_Store" in content:
        content.remove(".DS_Store")
    if content[0].endswith(ext):
        return [path+"/"+file for file in content]
    else:
        return np.concatenate([get_rec_files(path+"/"+subpath,ext) for subpath in content])

In [4]:
images = get_rec_files("data/images",".png")

In [5]:
images[:10]

array(['data/images/S111/007/S111_007_00000008.png',
       'data/images/S111/007/S111_007_00000009.png',
       'data/images/S111/007/S111_007_00000013.png',
       'data/images/S111/007/S111_007_00000007.png',
       'data/images/S111/007/S111_007_00000006.png',
       'data/images/S111/007/S111_007_00000012.png',
       'data/images/S111/007/S111_007_00000004.png',
       'data/images/S111/007/S111_007_00000010.png',
       'data/images/S111/007/S111_007_00000011.png',
       'data/images/S111/007/S111_007_00000005.png'], dtype='<U42')

In [6]:
emotions = get_rec_files("data/emotions",".txt")

In [7]:
emotions[:10]

array(['data/emotions/S111/007/S111_007_00000014_emotion.txt',
       'data/emotions/S111/001/S111_001_00000014_emotion.txt',
       'data/emotions/S111/006/S111_006_00000010_emotion.txt',
       'data/emotions/S129/012/S129_012_00000011_emotion.txt',
       'data/emotions/S129/006/S129_006_00000010_emotion.txt',
       'data/emotions/S129/011/S129_011_00000018_emotion.txt',
       'data/emotions/S129/002/S129_002_00000011_emotion.txt',
       'data/emotions/S116/007/S116_007_00000017_emotion.txt',
       'data/emotions/S116/001/S116_001_00000014_emotion.txt',
       'data/emotions/S116/006/S116_006_00000007_emotion.txt'],
      dtype='<U52')

In [8]:
def merge_groups(emotions,images):
    groups = []
    for emotion in emotions:
        emot_value = get_emot_value(emotion)
        prefix = emotion.split("/")[-1][:8]
        f=np.frompyfunc(lambda x: prefix in x, 1,1)
        selected_images = images[list(f(images))]
        selected_images.sort()
        groups += [(emot_value,selected_images)]
    return groups

In [9]:
def get_emot_value(path):
    with open(path,"r") as f:
        return f.read().split()[0][0]

In [10]:
group_images = merge_groups(emotions,images)

In [11]:
group_images[:2]

[('3', array(['data/images/S111/007/S111_007_00000001.png',
         'data/images/S111/007/S111_007_00000002.png',
         'data/images/S111/007/S111_007_00000003.png',
         'data/images/S111/007/S111_007_00000004.png',
         'data/images/S111/007/S111_007_00000005.png',
         'data/images/S111/007/S111_007_00000006.png',
         'data/images/S111/007/S111_007_00000007.png',
         'data/images/S111/007/S111_007_00000008.png',
         'data/images/S111/007/S111_007_00000009.png',
         'data/images/S111/007/S111_007_00000010.png',
         'data/images/S111/007/S111_007_00000011.png',
         'data/images/S111/007/S111_007_00000012.png',
         'data/images/S111/007/S111_007_00000013.png',
         'data/images/S111/007/S111_007_00000014.png'], dtype='<U42')),
 ('7', array(['data/images/S111/001/S111_001_00000001.png',
         'data/images/S111/001/S111_001_00000002.png',
         'data/images/S111/001/S111_001_00000003.png',
         'data/images/S111/001/S111_00

In [12]:
emot_count = [emot for emot,image in group_images]

In [13]:
Counter(emot_count)

Counter({'1': 45, '2': 18, '3': 59, '4': 25, '5': 69, '6': 28, '7': 83})

# Image Preparation

In [48]:
def prepare_images(images):
    faceCascade = cv2.CascadeClassifier("data/face_recog.xml")
    prepared_images = []
    
    for path in images:
        
        image = cv2.imread(path)
        
        faces = faceCascade.detectMultiScale(
            image,
            scaleFactor=1.1,
            minNeighbors=5,
            minSize=(48,48),
            flags = cv2.CASCADE_SCALE_IMAGE
        )
        for (x, y, w, h) in faces:
            crop_img = image[y:y+h, x:x+w]
            
        res_img = cv2.resize(cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY), (48,48)) 
        
        prepared_images += [(res_img,path)]
        
    return prepared_images

In [49]:
group_faces = []
for emot,pictures in group_images:
    group_faces += [(emot,prepare_images(pictures))]


In [50]:
group_faces[0]

('3', [(array([[44, 34, 21, ..., 48, 47, 49],
          [40, 11,  1, ..., 50, 49, 50],
          [10,  0,  0, ..., 40, 52, 52],
          ...,
          [28, 29, 28, ...,  0,  0,  0],
          [27, 30, 30, ...,  1, 14, 13],
          [26, 29, 28, ..., 35, 37, 24]], dtype=uint8),
   'data/images/S111/007/S111_007_00000001.png'),
  (array([[42, 36,  3, ..., 47, 48, 48],
          [36,  0,  1, ..., 48, 49, 49],
          [ 5,  1,  0, ..., 31, 49, 51],
          ...,
          [30, 30, 32, ...,  0,  0,  0],
          [29, 29, 27, ...,  0,  0,  3],
          [25, 29, 28, ..., 32, 30, 28]], dtype=uint8),
   'data/images/S111/007/S111_007_00000002.png'),
  (array([[42, 35, 26, ..., 47, 46, 49],
          [40, 14,  0, ..., 51, 49, 51],
          [12,  0,  0, ..., 36, 48, 50],
          ...,
          [28, 28, 30, ...,  0,  0,  0],
          [27, 31, 28, ...,  0, 13,  8],
          [26, 29, 29, ..., 35, 35, 28]], dtype=uint8),
   'data/images/S111/007/S111_007_00000003.png'),
  (array([[43, 34

In [55]:
paths = []
for emot,pictures in group_faces:
    for pixel,path in pictures:
        file_name = path.split("/")[-1]
        name = file_name.split(".")[0]
        
        face_path = "data/faces/"+name+"_"+emot
        np.save(face_path,pixel)
        paths += [(emot,face_path)]

In [23]:
for (x, y, w, h) in faces:
    cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)

In [170]:
filename = tf.constant(group_images[45][1][0])
image_string = tf.read_file(filename)
image_decoded = tf.image.decode_png(image_string)

In [171]:
with tf.Session() as session:
    pixel = session.run(image_decoded)

In [172]:
pixels = [178,125,119,135,67,80,45,81,84,69,71,20,51,69,42,35,41,44,97,135,152,161,162,164,167,167,131,134,114,90,76,71,56,35,57,67,42,25,34,27,25,20,34,37,76,96,99,111,172,142,120,94,38,60,56,44,49,77,52,27,48,54,55,63,64,134,162,169,170,172,174,172,172,168,129,149,125,87,83,72,56,46,45,53,35,33,55,21,22,23,23,25,77,80,81,57,163,163,119,53,60,58,76,59,45,44,35,43,42,43,48,64,120,168,173,177,177,182,184,183,180,173,160,132,96,118,132,100,49,38,39,43,41,51,37,25,18,22,18,16,40,98,54,22,152,138,111,79,82,89,75,79,48,33,43,35,28,21,52,113,165,176,181,181,184,187,188,193,193,185,185,141,91,113,139,138,80,49,37,29,57,51,50,46,26,29,17,20,12,73,69,67,153,111,95,74,62,68,95,55,44,62,49,41,47,54,115,164,177,183,186,186,190,193,198,198,195,190,192,165,133,124,109,103,81,53,30,32,42,26,44,53,29,17,21,16,19,54,35,71,71,84,96,78,66,43,41,39,62,62,36,38,39,101,159,171,179,186,187,191,193,197,199,196,190,189,184,188,150,134,125,102,63,41,26,30,30,33,32,17,13,18,43,15,22,24,30,71,110,123,103,112,117,74,44,39,73,53,31,18,63,143,164,174,182,186,191,195,193,190,194,195,193,190,183,175,151,119,95,98,81,47,29,25,24,30,23,16,11,14,17,15,18,16,24,40,119,111,122,139,124,65,39,56,68,43,23,22,106,154,166,180,184,187,193,189,185,191,200,205,203,193,171,164,167,119,52,40,54,68,40,14,18,26,22,10,9,21,21,24,24,20,31,37,125,127,121,127,119,56,42,52,43,28,17,56,136,161,174,186,186,191,195,187,189,211,213,208,204,202,186,184,179,97,39,66,47,30,14,17,20,23,15,12,23,61,55,32,31,17,34,47,149,135,123,134,122,68,58,43,30,21,22,106,147,171,182,190,184,184,192,191,196,213,199,192,188,151,123,111,111,54,65,91,76,58,14,22,16,15,26,33,39,39,33,29,31,14,20,47,149,145,123,127,105,72,58,42,36,6,42,135,151,167,182,194,176,168,175,188,193,189,181,184,109,59,92,140,149,63,79,73,53,68,40,24,26,26,36,25,41,59,27,14,13,24,34,40,142,139,136,111,80,59,53,34,67,21,55,103,90,78,86,149,176,157,160,184,191,183,182,141,79,125,170,184,180,81,74,77,57,39,32,19,35,29,20,19,40,69,34,27,21,18,36,42,137,131,120,107,71,61,36,33,59,40,52,136,162,155,112,77,146,162,158,180,198,185,176,142,140,159,164,175,182,117,68,75,78,30,25,33,29,16,14,17,14,11,9,45,52,14,30,48,129,115,101,81,61,46,31,57,49,30,88,159,170,187,188,144,118,142,160,183,193,172,149,142,162,133,113,124,136,125,83,99,80,40,43,63,46,19,13,13,11,11,5,32,58,31,24,26,104,76,78,68,57,37,22,49,48,29,86,148,158,170,179,184,168,115,142,188,178,165,157,147,125,84,71,76,81,75,74,95,68,49,41,54,50,29,16,10,11,13,14,20,44,26,10,23,104,79,75,48,63,45,21,47,74,28,79,132,129,130,124,114,133,151,168,177,172,169,162,171,140,119,116,120,134,121,87,88,84,76,45,51,44,41,30,11,15,15,18,20,18,14,26,31,103,71,60,44,42,47,41,48,64,32,79,85,56,76,99,116,142,171,179,179,181,176,166,172,169,164,153,144,149,157,129,110,114,102,64,55,63,44,35,42,29,41,25,20,30,54,28,19,115,92,75,54,36,37,47,47,43,42,73,62,87,134,142,151,169,178,176,179,182,181,179,180,177,172,174,169,170,156,156,136,127,86,86,50,60,75,55,31,35,53,26,47,99,55,10,12,122,85,80,53,41,46,39,35,37,47,93,120,139,149,160,170,180,190,193,180,175,174,166,186,191,169,166,175,178,152,156,152,150,96,91,91,78,104,56,15,26,38,35,63,76,29,20,13,137,95,93,76,50,56,58,33,31,57,139,169,175,176,174,168,183,194,195,178,175,174,164,141,177,203,195,195,199,163,154,172,154,137,101,102,83,66,39,21,35,29,30,56,41,10,13,20,132,113,67,60,64,58,44,35,21,66,161,170,176,177,174,147,168,172,169,141,134,152,156,147,133,167,202,207,214,201,149,147,148,133,132,89,115,68,18,25,27,41,28,42,27,15,24,25,128,117,69,68,73,56,32,35,21,81,174,173,177,175,160,114,119,164,122,46,20,58,107,155,181,128,166,191,192,185,175,156,129,101,119,102,139,94,34,20,26,26,27,28,20,31,58,24,140,123,81,58,45,54,31,35,33,105,176,173,171,165,149,110,78,130,116,123,143,148,154,178,193,170,123,156,169,176,180,168,173,142,145,130,131,101,33,12,31,23,21,20,23,21,37,38,102,91,89,75,63,58,38,29,29,129,172,169,168,151,153,111,89,97,106,160,176,177,186,193,198,193,135,117,167,172,174,159,161,148,162,154,135,104,23,8,29,52,15,14,26,12,23,25,119,88,92,83,50,40,37,27,56,139,167,168,165,138,159,139,96,102,144,179,203,210,204,200,198,196,190,126,117,166,168,163,151,145,153,154,143,111,21,9,38,82,47,17,40,29,22,21,163,91,63,45,31,36,42,35,53,137,159,158,147,127,156,171,162,160,172,180,199,210,191,184,192,193,192,185,111,118,156,162,150,137,149,157,150,116,28,12,45,51,73,44,23,30,41,37,187,93,101,50,36,44,51,33,36,126,152,149,122,127,160,139,106,98,103,109,118,99,96,97,118,155,181,184,173,105,131,159,158,137,140,159,147,119,38,25,35,28,37,55,17,21,31,45,223,149,95,77,40,48,70,35,23,93,146,136,106,138,139,81,74,112,155,153,186,172,167,159,122,81,122,163,173,149,116,162,165,157,134,152,141,119,39,37,36,36,26,45,39,20,42,48,198,109,77,100,35,38,51,43,25,62,141,132,113,136,103,68,176,215,239,232,240,236,232,221,191,134,92,92,145,152,127,159,168,169,144,137,139,117,40,22,29,30,21,30,43,26,33,35,144,78,57,44,35,43,39,38,36,43,135,151,131,117,78,99,233,241,247,244,249,225,220,227,224,217,176,79,80,134,135,161,169,166,157,133,131,114,46,31,21,29,15,27,34,28,26,32,111,57,45,43,43,47,31,31,26,26,122,166,160,113,59,102,166,135,146,100,108,57,38,66,138,186,227,164,56,102,149,164,170,164,161,136,119,108,37,27,17,23,21,24,49,34,15,22,78,51,48,56,55,61,73,30,16,16,96,166,177,148,92,59,92,77,38,12,6,8,10,9,9,42,99,114,77,124,160,165,170,163,155,136,118,104,35,23,22,21,29,33,37,43,14,24,65,56,69,64,63,84,65,34,23,14,55,158,182,174,151,101,42,36,30,17,15,10,9,6,8,8,7,13,95,181,166,165,166,161,147,129,117,98,27,17,23,21,34,53,19,51,30,19,60,48,82,42,65,63,49,51,26,14,22,130,178,184,160,106,32,13,1,6,13,20,21,35,35,20,6,6,124,204,174,164,160,157,143,124,115,94,25,15,16,21,34,34,18,39,37,36,150,78,111,76,49,69,58,78,50,17,6,87,167,187,167,119,54,87,116,163,168,183,187,208,194,181,146,106,160,202,176,166,157,154,137,118,106,82,16,12,11,23,34,31,10,21,17,13,216,154,80,51,43,71,52,76,51,14,6,42,149,182,177,138,118,184,205,212,223,214,217,210,209,202,161,145,192,199,179,166,159,151,128,109,101,68,10,15,11,18,43,49,25,9,14,17,222,215,86,44,68,70,59,100,68,29,10,13,119,178,179,156,120,114,114,131,147,130,167,135,122,110,133,181,197,190,184,165,159,143,117,100,90,46,7,16,14,13,29,61,69,11,13,20,167,119,66,61,70,63,54,87,64,49,20,1,75,174,174,167,148,126,119,118,124,126,127,137,148,171,196,194,185,186,186,164,153,129,105,93,63,40,16,15,23,20,25,51,45,31,15,36,149,107,98,107,75,77,52,51,94,47,16,5,32,154,175,165,166,166,163,166,169,179,191,198,203,202,191,186,182,190,183,157,145,112,92,71,64,69,24,14,38,21,25,56,24,39,70,68,174,113,94,103,56,48,58,43,63,78,39,17,9,113,176,170,168,167,170,171,177,186,194,198,197,191,185,186,184,191,167,150,130,94,70,54,109,98,25,14,37,19,37,51,36,54,71,36,68,50,73,56,49,64,64,46,40,71,75,47,9,58,169,178,177,180,184,187,195,202,208,209,201,192,190,188,186,169,147,136,103,76,54,92,144,123,25,10,39,39,44,69,57,42,27,13,82,143,153,79,54,47,71,57,54,37,34,24,18,17,127,185,183,185,185,189,202,207,216,218,204,199,196,193,171,137,129,102,82,64,81,136,148,145,45,20,49,25,42,60,42,17,19,25,181,215,157,57,64,78,59,36,39,40,18,8,10,4,58,174,180,181,187,187,193,200,210,211,207,196,191,169,124,115,101,88,75,71,121,143,154,155,71,36,58,43,31,57,36,12,15,26,220,221,146,68,88,102,86,63,46,42,22,11,11,12,8,109,173,174,178,180,181,183,187,188,181,169,151,112,103,102,93,84,71,102,130,141,152,134,80,64,66,25,20,49,39,13,18,33,212,215,141,80,80,98,70,97,24,11,14,15,11,16,4,14,97,154,170,166,159,156,157,153,147,138,120,112,106,97,90,79,89,118,134,149,149,139,88,65,57,16,30,38,22,14,24,38,203,217,165,95,86,61,58,36,25,34,31,16,8,23,75,92,80,78,99,128,150,150,145,144,139,132,126,114,102,94,85,89,106,121,143,155,146,144,103,55,40,26,22,35,20,29,43,56,217,183,148,67,61,65,59,48,47,70,58,16,5,75,140,131,114,95,46,38,93,142,150,149,145,137,128,114,101,92,94,110,117,138,148,142,138,161,177,76,27,30,34,50,46,57,58,58,227,181,84,88,73,59,72,39,27,40,50,5,58,182,144,124,108,102,75,32,44,89,138,146,147,140,130,116,101,96,118,124,130,141,140,150,171,202,189,47,22,34,47,60,55,54,36,49]

(490, 640, 1)

# Model Definition

# Model Training